In [51]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [3]:
df = pd.read_csv("NMvW_data/v0.csv.gz", 
                 dtype=dict(Provenance="string", RelatedWorks="string"))

# TODO: save & load DF s.t. these lines are not necessary here                
df["ObjectID"] = df.ObjectID.astype("int")
df = df.set_index("ObjectID")

FileNotFoundError: [Errno 2] No such file or directory: 'NMvW_data/v0.csv.gz'

In [ ]:
df2 = df[(df.BeginISODate+df.EndISODate) > 0]

df2.apply(lambda r: len(str(r.Title) + str(r.Description)) > 6, axis="columns").sum()/df2.shape[0]

In [ ]:
type(df2.iloc[0].Description)

In [ ]:

import spacy
from nltk import Tree

nlp = spacy.load("nl_core_news_sm")

def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_
    
doc = nlp(df.Description.dropna().iloc[0])
print(doc)
[to_nltk_tree(sent.root).pretty_print() for sent in doc.sents]

In [ ]:
from spacy import displacy
displacy.render(doc, style='dep')

In [ ]:
[nlp(t) for t in tqdm(df.Description) if isinstance(t, str)]

---
# Search

In [ ]:
kws = "" # "hello, goodbye"
prep_kws = "|".join(kws.lower().replace(", ", ",").split(","))
        
if (not kws.strip()) or (not prep_kws):
    does_contain = [True]*df.shape[0]
else:
    print("what")
    
print(does_contain)

In [ ]:
from urllib.parse import unquote

unquote('"objectKeywords"%3A""%2C"objectStartDate"%3A""%2C"objectEndDate"')

---
# Dates

 1. make all dates at least 1 (because python cannot deal with years <= 0)  
    (or drop non-positive dates)  
    (drop objects with EndISODate <= 0 in any case (doesn't make any sense))  
 
 2. convert all year values to %Y-%m-%d format (expected by Werner's API)  
    (by using datetime.strptime; allows comparison with > )  

In [ ]:
from datetime import datetime
frmt = "%Y-%m-%d"
start_year, end_year = datetime.strptime("2011-02-01", frmt), datetime.strptime("2013-10-02", frmt)

In [ ]:
(start_year.year > df.BeginISODate).sum()

In [ ]:
df.BeginISODate[(df.BeginISODate > 1500)].hist(bins=100)

### 1.

In [ ]:
df = df[df.EndISODate > 0]

# drop
df = df[df.BeginISODate > 0]

# change
df["BeginISODate"] = df.BeginISODate.replace(0, 1)

### 2.

In [ ]:
df["start_date"] = df.BeginISODate.astype(str).str.zfill(4).apply(lambda s: dt.strptime(s, "%Y"))
df["end_date"] = df.EndISODate.astype(str).str.zfill(4).apply(lambda s: dt.strptime(s, "%Y"))

---
# Fixing Strange x-Axis Data Issue 

In [ ]:
https://sabio.diginfra.nl/api/v1/objects/<datasetID>/search/%7B%22objectKeywords%22%3A%22hengelo%22%2C%22objectStartDate%22%3A%22%22%2C%22objectEndDate%22%3A%22%22%2C%22objectParams%22%3A[%7B%22id%22%3A%22Classification%22%2C%22value%22%3A%22%22%7D%2C%7B%22id%22%3A%22Department%22%2C%22value%22%3A%22%22%7D]%2C%22engineId%22%3A%22RandomEnginev0%22%2C%22engineMinScore%22%3A0%2C%22engineMaxScore%22%3A1%2C%22engineParams%22%3A[%7B%22id%22%3A0%2C%22value%22%3A%22useless1%22%7D]%2C%22vocabularyTerms%22%3A%22bewindhebber%2Cbewindvoerder%2Cbomba%2Cbombay%2Ccimarron%2Cderde%20wereld%2Cdwerg%2Cexpeditie%2Cgouverneur%2Chalfbloed%2Chottentot%2Cinboorling%2Cindiaan%2Cindisch%2Cindo%2Cinheems%2Cinlander%2Cjap%2Cjappen%2Cjappenkampen%2Ckaffer%2Ckaffir%2Ckafir%2Ckoelie%2Ckolonie%2Clagelonenland%2Clandhuis%2Cmarron%2Cmarronage%2Cmissie%2Cmissionaris%2Cmoor%2Cmoors%2Cmoren%2Cmulat%2Coctroon%2Contdekken%2Contdekking%2Contdekkingsreis%2Contwikkelingsland%2Coorspronkelijk%2Coosters%2Copperhoofd%2Cori%C3%ABntaals%2Cpinda%2Cpolitionele%20actie%2Cprimitief%2Cprimitieven%2Cpygmee%2Cras%2Crasch%2Cslaaf%2Cstam%2Cstamhoofd%2Ctraditioneel%2Ctropisch%2Cwesters%2Cwilden%2Czendeling%2Czendelingen%2Czending%22%7D/scatterplot/1/405100?api=https%3A%2F%2Fsabio.diginfra.net%2Fapi%2Fv1%2F

In [ ]:
https://sabio.diginfra.nl/api/v1/engines

In [ ]:
https://sabio.diginfra.net/api/v1/objects/NMvW_v0/search?object_keywords=hengelo&object_start_date=&object_end_date=&engine_id=RandomEnginev0&engine_min_score=0&engine_max_score=1&vocabulary_terms=bewindhebber%2Cbewindvoerder%2Cbomba%2Cbombay%2Ccimarron%2Cderde%20wereld%2Cdwerg%2Cexpeditie%2Cgouverneur%2Chalfbloed%2Chottentot%2Cinboorling%2Cindiaan%2Cindisch%2Cindo%2Cinheems%2Cinlander%2Cjap%2Cjappen%2Cjappenkampen%2Ckaffer%2Ckaffir%2Ckafir%2Ckoelie%2Ckolonie%2Clagelonenland%2Clandhuis%2Cmarron%2Cmarronage%2Cmissie%2Cmissionaris%2Cmoor%2Cmoors%2Cmoren%2Cmulat%2Coctroon%2Contdekken%2Contdekking%2Contdekkingsreis%2Contwikkelingsland%2Coorspronkelijk%2Coosters%2Copperhoofd%2Cori%C3%ABntaals%2Cpinda%2Cpolitionele%20actie%2Cprimitief%2Cprimitieven%2Cpygmee%2Cras%2Crasch%2Cslaaf%2Cstam%2Cstamhoofd%2Ctraditioneel%2Ctropisch%2Cwesters%2Cwilden%2Czendeling%2Czendelingen%2Czending&object_param_Classification=&object_param_Department=&engine_param_0=useless1

---

In [ ]:
from src.results import Result

In [ ]:
param_names = ["Department", "Classification"]

In [ ]:
attributes = ["BeginISODate", "EndISODate"] 
attributes = attributes + param_names

In [ ]:
df[attributes]

In [ ]:
values = df[attributes]

In [ ]:
r = Result(param_names=param_names, rows=df, scores=np.zeros(len(df)), 
                 score_details=df, min_score=0., max_score=1.)

In [ ]:
r.values

---
# Keyword Search Dev

In [ ]:
kws = "Maarten Sint"
prep_kws = "|".join(kws.lower().replace(", ", ",").split(","))

In [ ]:
prep_kws

In [ ]:
d = dict(a=)

d.get("a", False)

In [ ]:
def f(**kwargs):
    print(kwargs)
    
    
d = dict(a=13)
f(a=0, **d)

---
# Integrating TypicalityEngine

In [1]:
from src.datasets import NMvW
from src.engines.TypicalityEnginev0 import Typicality, TypicalityEngine

NMvW.data = NMvW.data.iloc[:10000]

In [2]:
typE = TypicalityEngine(id_="TypicalityEnginev0",
                           name="TypicalityEngine/v0",
                           dataset=NMvW,
                           params=[],
                       from_saved=False)

(2, 3)-grams: Padding documents...: 100%|████████████████████████████████| 20000/20000 [00:00<00:00, 1353372.38it/s]


(2, 3)-grams: Term Document Matrix constructed...
(2, 3)-grams: Term frequencies precomputed...
(2, 3)-grams: Init done


(0, 1)-grams: Padding documents...: 100%|████████████████████████████████| 20000/20000 [00:00<00:00, 1945635.63it/s]


(0, 1)-grams: Term Document Matrix constructed...
(0, 1)-grams: Term frequencies precomputed...
(0, 1)-grams: Init done


100%|████████████████████████████████████████████████████████████████████████| 10000/10000 [00:10<00:00, 985.67it/s]


In [ ]:
texts = NMvW.data.sample(frac=0.3).fillna("")

typs, details = typE.score_and_detail(texts)

In [ ]:
import seaborn as sns
sns.histplot(typs, color="blue")

In [ ]:
details.iloc[-1]

In [ ]:
import joblib
joblib.dump(typE.typicality, "test.pkl")

In [ ]:
import joblib
joblib.dump(typE, "test.pkl")

---

In [3]:
import joblib
typ2 = joblib.load("test.pkl")

In [5]:
texts = NMvW.data.sample(frac=0.3).fillna("")
typs2, details2 = typ2.score_and_detail(texts)

In [7]:
typs, details = typE.score_and_detail(texts)

In [10]:
(typs == typs2)

3000

# Fast Iteration over Results

In [4]:
df = pd.read_csv("../../data/v0_2.csv.gz", 
                 dtype=dict(Provenance="string", RelatedWorks="string"))

# TODO: save & load DF s.t. these lines are not necessary here                
df["ObjectID"] = df.ObjectID.astype("int")
df = df.set_index("ObjectID")

In [46]:
ids = df.index.astype("string")
titles = df.Title.fillna("").astype("string")
thumbnails = pd.Series([""]*df.shape[0], 
                       name="Thumbnail",
                      index=df.index).astype("string")

In [47]:
r = pd.concat([titles, thumbnails], axis=1)
r.index, ids

(Int64Index([      2,       5,       8,       9,      10,      11,      14,
                  15,      16,      17,
             ...
             1140369, 1140378, 1140379, 1140380, 1140384, 1140385, 1140386,
             1140388, 1140389, 1153236],
            dtype='int64', name='ObjectID', length=146694),
 Index(['2', '5', '8', '9', '10', '11', '14', '15', '16', '17',
        ...
        '1140369', '1140378', '1140379', '1140380', '1140384', '1140385',
        '1140386', '1140388', '1140389', '1153236'],
       dtype='object', name='ObjectID', length=146694))

In [48]:
r#.astype("string")

,Title,Thumbnail
ObjectID,,
2,MANDJE,
5,?,
8,MAND,
9,DRAAGMAND,
10,NUMMER NIET IN GEBRUIK,
...,...,...
1140385,,
1140386,,
1140388,,


In [53]:
lens = []
for i, row in tqdm(r.to_dict("index").items()):
    lens.append(sum(map(len, row.values())))

100%|██████████████████████████████████████████████████████████████████| 146694/146694 [00:00<00:00, 1608636.24it/s]


In [57]:
lens = []
for i in tqdm(range(r.shape[0])):
    lens.append(sum(map(len, r.iloc[i])))

100%|████████████████████████████████████████████████████████████████████| 146694/146694 [00:09<00:00, 14677.50it/s]


In [74]:
r_ = r.apply(lambda col: col.str.replace(" ", "X"), axis=0)
r_.columns = [c + "X" for c in r_.columns]
r2 = pd.concat([r, r_], axis=1)

In [86]:
t = pd.Series(np.random.random(size=df.shape[0]), index=df.index)

In [112]:
t.iloc[0].round(3)

0.587